# Lab 5: Airflow Integration - Orchestrating dbt với Airflow

## 🎯 Objectives
- Tích hợp dbt với Airflow
- Sử dụng dbt operators trong Airflow
- Schedule dbt runs với Airflow
- Error handling và retries
- Best practices cho dbt + Airflow

## 📋 Prerequisites
- Hoàn thành Lab 1-4
- Airflow Lab đã hoàn thành
- Airflow cluster đang chạy
- dbt project đã setup

## 🏗️ Integration Overview

**Airflow + dbt** là combination mạnh mẽ:
- **Airflow**: Orchestration và scheduling
- **dbt**: Data transformations
- **Together**: Complete data pipeline

### Architecture:
```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Sources    │────▶│   Airflow   │────▶│     dbt     │
│  (Raw Data) │     │(Orchestrate) │     │(Transform)  │
└─────────────┘     └─────────────┘     └─────────────┘
                            │
                            ▼
                    ┌─────────────┐
                    │   Models    │
                    │ (Analytics) │
                    └─────────────┘
```


## 1. dbt Operators trong Airflow

Airflow có built-in operators cho dbt.


In [ ]:
print("🔧 dbt Operators trong Airflow:")
print("=" * 60)
print("""
1. DbtRunOperator
   - Chạy dbt run command
   - Usage:
     from airflow.providers.dbt.cloud.operators.dbt import DbtRunOperator
     
     dbt_run = DbtRunOperator(
         task_id='dbt_run',
         dbt_conn_id='dbt_default',
         project_dir='/path/to/dbt/project',
         profiles_dir='/path/to/profiles',
         select=['staging.*']
     )

2. DbtTestOperator
   - Chạy dbt test command
   - Usage:
     from airflow.providers.dbt.cloud.operators.dbt import DbtTestOperator
     
     dbt_test = DbtTestOperator(
         task_id='dbt_test',
         dbt_conn_id='dbt_default',
         project_dir='/path/to/dbt/project',
         profiles_dir='/path/to/profiles'
     )

3. DbtDocsGenerateOperator
   - Generate dbt documentation
   - Usage:
     from airflow.providers.dbt.cloud.operators.dbt import DbtDocsGenerateOperator
     
     dbt_docs = DbtDocsGenerateOperator(
         task_id='dbt_docs',
         dbt_conn_id='dbt_default',
         project_dir='/path/to/dbt/project'
     )

4. BashOperator (Alternative)
   - Chạy dbt commands qua bash
   - Usage:
     from airflow.operators.bash import BashOperator
     
     dbt_run = BashOperator(
         task_id='dbt_run',
         bash_command='cd /path/to/dbt/project && dbt run --profiles-dir .'
     )
""")
print("=" * 60)


## 2. Complete Airflow DAG với dbt

Tạo DAG hoàn chỉnh tích hợp dbt.


In [ ]:
# Example Airflow DAG với dbt
dag_example = """
from airflow.sdk import DAG, task
from airflow.operators.bash import BashOperator
from airflow.providers.standard.operators.empty import EmptyOperator
import pendulum

@dag(
    dag_id='dbt_pipeline',
    schedule='@daily',
    start_date=pendulum.datetime(2024, 1, 1, tz='UTC'),
    catchup=False,
    tags=['dbt', 'data-pipeline'],
)
def dbt_pipeline():
    start = EmptyOperator(task_id='start')
    
    # Run staging models
    dbt_run_staging = BashOperator(
        task_id='dbt_run_staging',
        bash_command='cd /path/to/dbt_lab && dbt run --select staging.* --profiles-dir . --project-dir .',
    )
    
    # Run marts models
    dbt_run_marts = BashOperator(
        task_id='dbt_run_marts',
        bash_command='cd /path/to/dbt_lab && dbt run --select marts.* --profiles-dir . --project-dir .',
    )
    
    # Run tests
    dbt_test = BashOperator(
        task_id='dbt_test',
        bash_command='cd /path/to/dbt_lab && dbt test --profiles-dir . --project-dir .',
    )
    
    # Generate docs
    dbt_docs = BashOperator(
        task_id='dbt_docs',
        bash_command='cd /path/to/dbt_lab && dbt docs generate --profiles-dir . --project-dir .',
    )
    
    end = EmptyOperator(task_id='end')
    
    # Define dependencies
    start >> dbt_run_staging >> dbt_run_marts >> dbt_test >> dbt_docs >> end

dbt_pipeline_instance = dbt_pipeline()
"""

print("📋 Complete Airflow DAG Example:")
print("=" * 60)
print(dag_example)
print("=" * 60)

print("\n💡 Key Points:")
print("  - Run staging models trước")
print("  - Run marts models sau")
print("  - Run tests để verify")
print("  - Generate docs cuối cùng")
print("  - Error handling với retries")


## 3. Error Handling và Retries

Best practices cho error handling.


In [ ]:
print("🛡️ Error Handling Best Practices:")
print("=" * 60)
print("""
1. Set retries cho dbt tasks:
   dbt_run = BashOperator(
       task_id='dbt_run',
       bash_command='...',
       retries=3,
       retry_delay=timedelta(minutes=5),
   )

2. Use on_failure_callback:
   def handle_dbt_failure(context):
       # Send alerts, cleanup, etc.
       pass
   
   dbt_run = BashOperator(
       task_id='dbt_run',
       bash_command='...',
       on_failure_callback=handle_dbt_failure,
   )

3. Check dbt exit codes:
   - 0: Success
   - 1: General error
   - 2: Parse error
   
   Use check=True trong BashOperator để fail on error

4. Store dbt logs:
   dbt_run = BashOperator(
       task_id='dbt_run',
       bash_command='cd /path/to/dbt && dbt run >> logs/dbt_run.log 2>&1',
   )

5. Conditional execution:
   - Skip downstream tasks nếu tests fail
   - Use BranchPythonOperator
   - Check test results trước khi continue
""")
print("=" * 60)


## 4. Best Practices

Best practices cho dbt + Airflow integration.


In [ ]:
print("✅ Best Practices:")
print("=" * 60)
print("""
1. **Separation of Concerns:**
   - Airflow: Orchestration và scheduling
   - dbt: Data transformations
   - Don't mix logic

2. **Task Granularity:**
   - Separate tasks cho staging, marts, tests
   - Easier to debug và rerun
   - Better visibility trong Airflow UI

3. **Dependencies:**
   - Run staging → marts → tests
   - Use Airflow dependencies properly
   - Don't run tests nếu models fail

4. **Error Handling:**
   - Set appropriate retries
   - Handle dbt failures gracefully
   - Send alerts cho critical failures

5. **Logging:**
   - Store dbt logs
   - Use Airflow logs
   - Make debugging easier

6. **Performance:**
   - Run models in parallel khi có thể
   - Use dbt selectors efficiently
   - Monitor execution time

7. **Testing:**
   - Always run tests sau models
   - Fail fast nếu tests fail
   - Don't proceed nếu data quality issues

8. **Documentation:**
   - Generate docs trong pipeline
   - Keep docs up-to-date
   - Share với team
""")
print("=" * 60)


## 5. Tóm tắt và Kết luận

### ✅ Những gì đã học trong toàn bộ dbt Lab Series:

**Lab 1: dbt Basics**
- dbt là gì và tại sao sử dụng
- dbt project structure
- dbt commands cơ bản

**Lab 2: Models và SQL Transformations**
- Staging và marts models
- Jinja templating
- Materializations

**Lab 3: Testing và Documentation**
- Generic và custom tests
- Documentation
- Source freshness

**Lab 4: Macros và Jinja**
- Custom macros
- Advanced Jinja
- dbt_utils package

**Lab 5: Airflow Integration**
- Tích hợp dbt với Airflow
- dbt operators
- Error handling

### 🎯 Key Takeaways:

**dbt là gì:**
- Data transformation tool
- SQL với Jinja templating
- Version control cho analytics
- Testing và documentation

**dbt + Airflow:**
- Airflow orchestrates
- dbt transforms
- Together = Complete pipeline

**Best Practices:**
- Separation of concerns
- Proper dependencies
- Error handling
- Testing và documentation

### 📚 Next Steps:

1. **Production Deployment:**
   - Setup dbt trên production
   - Configure Airflow cho production
   - Monitoring và alerting

2. **Advanced Topics:**
   - dbt Cloud
   - Python models trong dbt
   - Snapshots và seeds
   - Multi-environment workflows

3. **Integration:**
   - Cloud data warehouses
   - CI/CD cho dbt
   - Data quality tools
   - BI tools integration

### 🔗 Useful Links:
- [dbt Documentation](https://docs.getdbt.com/)
- [Airflow dbt Provider](https://airflow.apache.org/docs/apache-airflow-providers-dbt/)
- [dbt Best Practices](https://docs.getdbt.com/guides/best-practices)

### 🎉 Congratulations!

Bạn đã hoàn thành toàn bộ dbt Lab Series! Bây giờ bạn có đủ kiến thức để:
- Thiết kế và implement dbt projects
- Transform data với dbt
- Test và document data pipelines
- Tích hợp dbt với Airflow
- Deploy và maintain production pipelines

**Happy Transforming! 🚀**
